In [67]:
import numpy as np
import matplotlib.pyplot as plt

In [68]:
# Função que soluciona Ax = b 
def soluciona_sistema(A, b):
    # Verificar se a matriz A é quadrada
    if A.shape[0] != A.shape[1]:
        raise ValueError("A matriz A deve ser quadrada.")
    
    # Verificar se o número de colunas de A é igual ao número de elementos em b
    if A.shape[1] != len(b):
        raise ValueError("O número de colunas de A deve ser igual ao número de elementos em b.")
    
    # Concatenar a matriz A com o vetor b
    Ab = np.column_stack((A, b))
    
    # Aplicar o escalonamento
    n = Ab.shape[0]
    for i in range(n):
        # Encontrar o pivô
        pivot = Ab[i, i]
        if pivot == 0:
            raise ValueError("O pivô é zero. O sistema não tem solução única.")
        
        # Normalizar a linha do pivô
        Ab[i] = np.array(Ab[i] / pivot)
        
        # Eliminar as entradas abaixo do pivô
        for j in range(i + 1, n):
            multiplier = Ab[j, i]
            Ab[j, :] -= multiplier * Ab[i, :]
    
    # Resolver o sistema triangular superior resultante
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = Ab[i, -1]
        for j in range(i + 1, n):
            x[i] -= Ab[i, j] * x[j]
    
    return x

In [69]:
def coeficientes_trajetoria(A_):
    """Determina a trajetoria que melhor aproxima a feita por um objeto
    celeste a partir de um conjunto de medicoes.

    INPUT
    -----
    A : array_like de dimensao 2
        Posicoes do objeto celeste. 

    OUTPUT
    ------
    coefs : array_like de ordem 6
        Coeficientes [A,B,C,D,E,F] da conica que melhor aproxima a trajetoria
        do objeto celeste no plano xy, por uma equacao do tipo
            Ax^2 + 2Bxy + Cy^2 + 2Dx + 2Ey + F = 0.
    """
    
    # Copia a matriz para evitar erros
    A = A_.copy()

    # Calcula o menor valor (em módulo do array)
    min = np.amin(abs(A))

    # Reescala os dados
    A_reescalada = A/min

    linha = lambda x, y : [x*x, 2*x*y, y*y, 2*x, 2*y] 

    # Dc=b, para F=1:
    # Ax² + 2Bxy + Cy² + 2Dx + 2Ey = -1

    D = []
    for i in range(0, len(A_reescalada)):
        D.append(linha(A_reescalada[i,0], A_reescalada[i,1]))

    # Define a matriz b como uma matriz: [-1, -1, ....., -1]
    b = -np.ones(len(D))

    # Calcula a transposta
    D_t = np.transpose(D)

    # Multiplica os dois lados da equação pela transposta de D
    A = D_t@D
    b = D_t@b

    # calcula a matriz dos coeficientes
    coefs = soluciona_sistema(A, b)

    # Adiciona o valor de F na matriz dos coeficientes
    coefs = np.append(coefs, 1, axis=None)

    return coefs

In [70]:
def tipo_trajetoria(A):
    """Determina o tipo da trajetoria que melhor aproxima a feita por um objeto
    celeste a partir de um conjunto de medicoes.

    INPUT
    -----
    A : array_like de dimensao 2
        Posicoes do objeto celeste. 

    OUTPUT
    ------
    tipo_trajetoria : inteiro (0, 1 ou 2)
        Tipo de trajetória.
            0: Trajetoria eliptica
            1: Trajetoria parabolica
            2: Trajetoria hiperbolica
    """
    tol = 2*10**(-9)

    A, B, C, D, E, F = coeficientes_trajetoria(A)

    det = A*C - B*B

    if (det > tol):
        # print("Eliptica")
        return 0
    elif (-det > tol):
        # print("Parabolica")
        return 1
    else:
        # print("Hiperbolica")
        return 2

In [71]:
tol = 2*10**(-9)

elipses = np.load("teste_e.npy")
array_elipses = []
i = 0

for obj in elipses:
    A, B, C, D, E, F = coeficientes_trajetoria(obj)
    det = A*C - B*B
    array_elipses.append(det)

    if abs(det) < tol:
        i+=1

print("Elipses")
print(f"Lidos certo: {100-i}/{len(elipses)}")




hiperboles = np.load("teste_h.npy")
array_hiperboles = []
i = 0

for obj in hiperboles:
    A, B, C, D, E, F = coeficientes_trajetoria(obj)
    det = A*C - B*B

    array_hiperboles.append(det)

    if abs(det) < tol:
        i+=1
array_elipses.sort()
print(f"\nHipérboles")
print(max(array_hiperboles))
print(min(array_hiperboles))
print(f"Lidos certo: {100-i}/{len(hiperboles)}")

# print(f"\n\n\n")
# print(array_elipses)
# print(f"\n\n\n")

parabulas = np.load("teste_p.npy")
array_parabulas = []
i = 0

for obj in parabulas:
    A, B, C, D, E, F = coeficientes_trajetoria(obj)
    det = A*C - B*B
    array_parabulas.append(det)

    if abs(det) > tol:
        i+=1


print(f"\nParabulas")
print(max(array_parabulas))
print(min(array_parabulas))
print(f"Lidos certo: {100-i}/{len(parabulas)}")

Elipses
Lidos certo: 79/100

Hipérboles
-1.6371400904369927e-17
-0.002749512263211085
Lidos certo: 79/100

Parabulas
2.7656048583328367e-05
-1.442929906211472e-06
Lidos certo: 77/100


In [72]:
elipses = np.load("teste_e.npy")
i1, i2, i3 = 0, 0, 0

for obj in elipses:
    tipo_objeto = tipo_trajetoria(obj)
    if tipo_objeto == 0:
        i1+=1
    elif tipo_objeto == 1:
        i2+=1
    else:
        i3+=1

print("i1: ", i1)
print("i2: ", i2)
print("i3: ", i3)
    

# print("Elipses")
# print(max(array_elipses))
# print(min(array_elipses))
# print(f"Lidos errados: {i}/{len(elipses)}")

i1:  79
i2:  0
i3:  21
